In [25]:
import pandas as pd
import re

# Define the file path for the CSV and the output path
file_path = r"C:\Users\avina\Downloads\Connections.csv"
output_path = r"C:\Users\avina\Downloads\Cleaned_Connections.csv"

data = pd.read_csv(file_path, encoding="utf-8", on_bad_lines="warn")
    


In [27]:
data.head(3)

,First Name,URL,Email Address,Company,Position,Connected On
0,Pranjali,https://www.linkedin.com/in/pranjali-modi-1746...,NaN,Cognizant,Programmer Analyst,15-Nov-24
1,Allaka,https://www.linkedin.com/in/allaka-rao-58957017,NaN,upGrad,Corporate Relations Manager,15-Nov-24
2,Sairaj,https://www.linkedin.com/in/sairaj-rakshale,NaN,CodSoft,Machine Learning Intern,15-Nov-24


In [91]:
# Define full decision-maker titles (expanded from short forms)
decision_maker_titles = [
    "Chief Executive Officer", "Chief Visionary Officer", "Chief Marketing Officer", "Founder", "Co-founder", 
    "Head", "Managing Director", "Chief Officer", "President", "Owner", "Managing Director", 
    "Executive Director", "Co-Founder", "Principal", "Owner", "Chief","Operations Manager","Divisional","Country","India",
     "Procurement Manager","Chief Technology Officer", "Chief Operating Officer", "Chief Financial Officer", 
    "Chief Strategy Officer", "Chief Human Resources Officer","Medical"
]

# Define non-decision-maker titles to exclude
non_decision_maker_titles = [
    "Data Analyst", "Software Developer", "Software Engineer", "Engineer", "Developer", 
    "Intern", "Junior", "Assistant", "Technician", "Support", "Tester", "Analyst"
]

# Load the data
data = pd.read_csv(file_path, encoding="utf-8", on_bad_lines="warn")

# Filter the rows where 'Position' contains any decision maker title and exclude non-decision-maker titles
df= data[
    data['Position'].str.contains('|'.join(decision_maker_titles), case=False, na=False) & 
    ~data['Position'].str.contains('|'.join(non_decision_maker_titles), case=False, na=False)
]


In [93]:
# Store the filtered data into another DataFrame (filtered_data already holds it)
df.head(5)  # Display the first few rows of the filtered data

,First Name,URL,Email Address,Company,Position,Connected On
11,Vikram,https://www.linkedin.com/in/vikramchallur,NaN,MakeMyTrip,Vice President - Hotel Supply Network,05-Nov-24
23,Tamara,https://www.linkedin.com/in/tamara-mamman-8894...,NaN,The Nomad,Small Business Owner / Entrepreneur,26-Oct-24
44,Manoj,https://www.linkedin.com/in/manoj-ahlawat-6a451b5,NaN,Max Healthcare,Vice President,16-Oct-24
45,Vineesh,https://www.linkedin.com/in/vineesh-ghei-98616249,NaN,HealthCare Global,Chief Sales Officer (CSO),16-Oct-24
49,Madan,https://www.linkedin.com/in/madan-sampath-%F0%...,NaN,HealthCare Global,Chief Supply Chain Officer & VP,12-Oct-24


In [95]:
output_excel_path = r"C:\Users\avina\Downloads\Filtered_Decision_Makers.xlsx"

df.to_excel(output_excel_path, index=False)

print(f"Filtered decision makers saved as {output_excel_path}")

Filtered decision makers saved as C:\Users\avina\Downloads\Filtered_Decision_Makers.xlsx


In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Define file paths
input_excel_path = r"C:\Users\avina\Downloads\Filtered_Decision_Makers.xlsx"
output_excel_path_bangalore = r"C:\Users\avina\Downloads\Profiles_Bangalore.xlsx"
output_excel_path_outside = r"C:\Users\avina\Downloads\Profiles_Outside_Bangalore.xlsx"

# Load the Excel file
data = pd.read_excel(input_excel_path)

# Extract URLs from the Excel file
urls = data['URL']

# Configure Chrome options for incognito mode
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

# Initialize Selenium WebDriver with incognito mode
driver = webdriver.Chrome(options=chrome_options)

# Log in to LinkedIn
driver.get("https://www.linkedin.com/login")
time.sleep(3)

# Add the cookie for authentication
cookie = {
    "name": "li_at",
    "value": "AQEDAVUFlgcDZWCtAAABk0WjpnUAAAGTabAqdU0AeaV74jJHfl62jv9Q1KShKYLdF0R8y77gtRoq346ZRrRAuDTRur5W8IY_JMXAJyV-ptY9uQRcWBVHB93ggEhCb4AMMZ6-nxtooEPbcLaoNdbXK6aq",
    "domain": ".linkedin.com"
}
driver.add_cookie(cookie)
driver.refresh()
time.sleep(3)

# Prepare lists to store results
bangalore_profiles = []
outside_bangalore_profiles = []

# Iterate over each URL
for url in urls:
    try:
        driver.get(url)
        time.sleep(5)  # Wait for the page to load

        # Scrape location using the provided XPath
        location_element = driver.find_element(By.XPATH, '//*[@id="profile-content"]/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[1]')
        location = location_element.text

        # Classify based on location
        if "Bangalore" in location:
            bangalore_profiles.append((url, location))
        else:
            outside_bangalore_profiles.append((url, location))

    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Close the driver
driver.quit()

# Convert results into DataFrames
df_bangalore = pd.DataFrame(bangalore_profiles, columns=["URL", "Location"])
df_outside = pd.DataFrame(outside_bangalore_profiles, columns=["URL", "Location"])

# Save results to Excel files
df_bangalore.to_excel(output_excel_path_bangalore, index=False)
df_outside.to_excel(output_excel_path_outside, index=False)

print("Scraping completed and results saved.")


Error processing URL https://www.linkedin.com/in/shivang-joshi-82b41920b: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="profile-content"]/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[1]"}
  (Session info: chrome=131.0.6778.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E5996CB5+28821]
	(No symbol) [0x00007FF7E5903840]
	(No symbol) [0x00007FF7E57A578A]
	(No symbol) [0x00007FF7E57F91BE]
	(No symbol) [0x00007FF7E57F94AC]
	(No symbol) [0x00007FF7E5842647]
	(No symbol) [0x00007FF7E581F33F]
	(No symbol) [0x00007FF7E583F412]
	(No symbol) [0x00007FF7E581F0A3]
	(No symbol) [0x00007FF7E57EA778]
	(No symbol) [0x00007FF7E57EB8E1]
	GetHandleVerifier [0x00007FF7E5CCFCAD+3408013]
	GetHandleVerifier [0x00007FF7E5CE741F+3504127]
	GetHandleVerifier [0x00007FF7E5CDB5FD+3455453]
	GetHandleVerifier [0x00